In [1]:
# import the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F

from einops import rearrange, repeat
from einops.layers.torch import Rearrange
import numpy as np

In [2]:
data_path = '../data/VTT_ConIot_Dataset'
IMU_path = data_path + '/IMU'
Keypoint_path = data_path + '/Keypoint'


In [3]:
# Load activity 1 for user 1
keypoint_data = pd.read_csv(Keypoint_path + '/Subject_01_Task_1.m2ts_keyPoints.csv')
imu_data = pd.read_csv(IMU_path + '/activity_1_user_1_combined.csv')

In [4]:
print (keypoint_data.head())
print (imu_data.head())

print(keypoint_data.shape)
print(imu_data.shape)

print(keypoint_data.columns)
print(imu_data.columns)

print(keypoint_data.describe())
print(imu_data.describe())

   frame_number  timestamp       x0       y0   prob0       x1       y1  \
0             0          0  909.797  232.566  0.3156  912.662  218.176   
1             1         40  902.369  264.382  0.2932  908.107  247.104   
2             2         80  909.693  276.270  0.6559  911.127  263.325   
3             3        120  913.709  293.351  0.7699  915.149  280.400   
4             4        160  918.769  296.528  0.6209  920.208  283.577   

    prob1       x2       y2  ...      x14      y14  prob14      x15      y15  \
0  0.3663  895.469  221.054  ...  816.663  706.000  0.1226  861.081  851.340   
1  0.4096  888.024  252.863  ...  817.733  715.061  0.1192  849.293  856.168   
2  0.5649  892.488  264.764  ...  832.272  727.915  0.1609  848.043  854.490   
3  0.6624  896.434  281.839  ...  841.731  735.124  0.1302  850.368  856.000   
4  0.7260  902.934  285.016  ...  838.156  741.175  0.1317  849.672  854.856   

   prob15      x16      y16  prob16  detection_score  
0  0.2170  786.574 

In [5]:
# for column date time in imu_data, change to time stamp starting from 0
# first remove the "0 days " substring from the datetime column strings
imu_data['datetime'] = imu_data['datetime'].str.replace('0 days ', '')
imu_data['datetime'] = pd.to_datetime(imu_data['datetime'])
# convert to milliseconds
imu_data['datetime'] = imu_data['datetime'].astype(np.int64)/1000000
# round off and convert to int
imu_data['datetime'] = imu_data['datetime'].round()
# keep only difference staring from 0
imu_data['datetime'] = imu_data['datetime'] - imu_data['datetime'].min()


C:\Users\Azhar\AppData\Local\Temp\ipykernel_6972\500998939.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  imu_data['datetime'] = pd.to_datetime(imu_data['datetime'])


In [6]:
print(imu_data['datetime'].head(100))

0       0.0
1      10.0
2      20.0
3      30.0
4      40.0
      ...  
95    950.0
96    960.0
97    970.0
98    980.0
99    990.0
Name: datetime, Length: 100, dtype: float64


In [7]:
print(imu_data['datetime'].max())
print(keypoint_data['timestamp'].max())

59980.0
59840


In [8]:
print(imu_data['datetime'].shape)
print(keypoint_data['timestamp'].shape)

# NOTE TO SELF: The timestamps are not the same. Simple solution would be to ignore the extra time stamps once they are brought dow nto the same frequency.

(5999,)
(1497,)


In [10]:
# check for any non numeric values in the imu data
print(imu_data.dtypes)

datetime            float64
trousers_index      float64
trousers_Gx_dps     float64
trousers_Gy_dps     float64
trousers_Gz_dps     float64
trousers_Ax_g       float64
trousers_Ay_g       float64
trousers_Az_g       float64
trousers_ts         float64
trousers_Mx_uT      float64
trousers_My_uT      float64
trousers_Mz_uT      float64
trousers_mbar       float64
trousers_tot_g      float64
trousers_tot_dps    float64
back_index          float64
back_Gx_dps         float64
back_Gy_dps         float64
back_Gz_dps         float64
back_Ax_g           float64
back_Ay_g           float64
back_Az_g           float64
back_ts             float64
back_Mx_uT          float64
back_My_uT          float64
back_Mz_uT          float64
back_mbar           float64
back_tot_g          float64
back_tot_dps        float64
hand_index          float64
hand_Gx_dps         float64
hand_Gy_dps         float64
hand_Gz_dps         float64
hand_Ax_g           float64
hand_Ay_g           float64
hand_Az_g           